In [ ]:
# import des packages 
import sys
sys.path.append('../')
from tools.packages import *

In [ ]:
# fonctions utiles pour la simulation des données
def create_covariates(p, n, step):
    covariance = np.repeat(0.5, p*p).reshape((p, p))
    covariance[np.eye(covariance.shape[0], dtype=bool)] = 1
    X = np.random.multivariate_normal(mean=np.repeat(0, p), cov=covariance, size=n)
    coeff = np.zeros(p)
    if step is not None:
        coeff[step] = 1
    return (X, coeff)

def create_dictionnary(p_ind, p_group):
    dict_group = dict()
    step = int(p_ind/p_group)
    for group in range(p_group):    
        for ind in range(group*step, group*step+step):
            dict_group.update({ind : group})
    return (dict_group)

In [ ]:
# paramètres du SMC
N = 10000
P = 1000
nruns = 5
nprocs = -5

In [ ]:
# simulation des données
p_group = 5
p_ext = 5

In [ ]:
settings = ((100, 50, 'ALA', 0.5), (200, 50, 'ALA', 0.5), (300, 50, 'ALA', 0.5), (400, 50, 'ALA', 0.5), 
            (500, 50, 'ALA', 0.5), (600, 50, 'ALA', 0.5), (700, 50, 'ALA', 0.5), (800, 50, 'ALA', 0.5),
            (900, 50, 'ALA', 0.5), (1000, 50, 'ALA', 0.5), (1250, 50, 'ALA', 0.5), (1500, 50, 'ALA', 0.5), 
            (1750, 50, 'ALA', 0.5), (2000, 50, 'ALA', 0.5), (2250, 50, 'ALA', 0.5), (2500, 50, 'ALA', 0.5),
            (1500, 10, 'ALA', 0.5), (1500, 25, 'ALA', 0.5), (1500, 75, 'ALA', 0.5),
            (1500, 100, 'ALA', 0.5), (1500, 125, 'ALA', 0.5), (1500, 150, 'ALA', 0.5), (1500, 175, 'ALA', 0.5),
            (1500, 200, 'ALA', 0.5), (1500, 225, 'ALA', 0.5), (1500, 250, 'ALA', 0.5),
            (100, 50, 'LA', 0.5), (200, 50, 'LA', 0.5), (300, 50, 'LA', 0.5), (400, 50, 'LA', 0.5), 
            (500, 50, 'LA', 0.5), (600, 50, 'LA', 0.5), (700, 50, 'LA', 0.5), (800, 50, 'LA', 0.5),
            (900, 50, 'LA', 0.5), (1000, 50, 'LA', 0.5), (1250, 50, 'LA', 0.5), (1500, 50, 'LA', 0.5), 
            (1750, 50, 'LA', 0.5), (2000, 50, 'LA', 0.5), (2250, 50, 'LA', 0.5), (2500, 50, 'LA', 0.5),
            (1500, 10, 'LA', 0.5), (1500, 25, 'LA', 0.5), (1500, 75, 'LA', 0.5),
            (1500, 100, 'LA', 0.5), (1500, 125, 'LA', 0.5), (1500, 150, 'LA', 0.5), (1500, 175, 'LA', 0.5),
            (1500, 200, 'LA', 0.5), (1500, 225, 'LA', 0.5), (1500, 250, 'LA', 0.5))

In [ ]:
results = []
for setting in settings:
    # setting
    n, p_ind, likelihood = setting[:3]
    pi_ind = pi_group = setting[3:][0]
    step = int(p_ind/p_group)
    
    # simulation des données
    X_ind, coeff_ind = create_covariates(p=p_ind, n=n, step=np.array([p_ind-(step*2+1), p_ind-(step+1), p_ind-1]))  
    X_group, coeff_group = create_covariates(p=p_group, n=n, step=np.array([2, 3, 4]))
    X_ext, coeff_ext = create_covariates(p=p_ext, n=n, step=np.array([2, 3, 4]))

    # concaténation des variables
    X = np.concatenate([X_ext, X_group, X_ind], axis=1)
    coeff = np.concatenate([coeff_ext, coeff_group, coeff_ind], axis=0)

    # construction de l'outcome
    y = np.matmul(X, coeff) + norm.rvs(size=n)
    y = np.where(y>=0, 1, 0)

    # dictionnaire des correspondances groupes-variables individuelles 
    dict_group = create_dictionnary(p_ind=p_ind, p_group=p_group)

    data = (X, y, dict_group, p_ext)

    # spécifier la loi a priori
    prior = dists.BiLevelPrior(p_group=p_group, p_ind=p_ind, pi_group=pi_group, pi_ind=pi_ind, dict_group=dict_group)

    # spécifier la méthode d'approximation de la vraisemblance marginale
    if likelihood == "ALA":
        model = bin.BilevelALA(data=data, prior=prior)
    elif likelihood == "LA":
        model = bin.BilevelLA(data=data, prior=prior)
    else:
        print("likelihood approximation method misspecified")
        break

    # spécifier la méthode MCMC
    mcmc = ssps.MCMCSequenceWF(mcmc=bin.BiLevelBinaryMetropolis(data=data), len_chain=P)

    # spécifier le modèle SMC 
    smc = ssps.AdaptiveTempering(model=model, move=mcmc, len_chain=P)

    # lancer le SMC
    start = time.time()
    output = particles.multiSMC(fk=smc, N=N // P, nruns=nruns, nprocs=nprocs, verbose=0)
    end = time.time()
    
    # sauvegarde du temps de calcul
    results.append({"n":n, "p_ind":p_ind, "likelihood":likelihood, 
                  "time":round((end-start)/60, 2), "prior":pi_ind})
    pd.DataFrame(results).to_csv("results_simulation/results_SMC.csv", index=False, sep=";")
    
    # analyse des variables sélectionnées
    print("n = {} / p_ind = {} / likelihood = {} / prior = {}".format(n, p_ind, likelihood, pi_ind))
    selection = np.mean(output[0]["output"].X.theta, axis=0)
    print("sélection des groupes")
    print(selection[:p_group])
    print("sélection des variables individuelles")
    print(selection[p_group:])
    print("------------------------------------------------------")
    print(" ")
    
    # sauvegarde du modèle
    filename = open("results_simulation/output_SMC_n_{}_p_{}_lik_{}_prior_{}.pkl".format(n, p_ind, likelihood, pi_ind), "wb")
    pickle.dump(output, filename)
    filename.close()